# QrackCircuit

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
width = 24

In [2]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install pyqrack

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

`QrackCircuit` is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [3]:
import math
import random

sqrt1_2 = 1 / math.sqrt(2)

def x_to_y(circ, q):
    circ.mtrx([1, 0, 0, 1j], q)
    return 1

def x_to_z(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def y_to_z(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 2

def y_to_x(circ, q):
    circ.mtrx([1, 0, 0, -1j], q)
    return 1

def z_to_x(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    return 1

def z_to_y(circ, q):
    circ.mtrx([sqrt1_2, sqrt1_2, sqrt1_2, -sqrt1_2], q)
    circ.mtrx([1, 0, 0, 1j], q)
    return 2

def cx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 1)
    return 1

def cy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 1)
    return 1

def cz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    return 1

def acx(circ, q1, q2):
    circ.ucmtrx([q1], [0, 1, 1, 0], q2, 0)
    return 1

def acy(circ, q1, q2):
    circ.ucmtrx([q1], [0, -1j, 1j, 0], q2, 0)
    return 1

def acz(circ, q1, q2):
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 0)
    return 1

def iswap(circ, q1, q2):
    circ.swap(q1, q2)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    circ.mtrx([1, 0, 0, 1j], q1)
    circ.mtrx([1, 0, 0, 1j], q2)
    return 4

def iiswap(circ, q1, q2):
    circ.mtrx([1, 0, 0, -1j], q2)
    circ.mtrx([1, 0, 0, -1j], q1)
    circ.ucmtrx([q1], [1, 0, 0, -1], q2, 1)
    circ.swap(q1, q2)
    return 4

def random_circuit(width, circ):
    gate_count = 0
    
    single_bit_gates = { 0: (z_to_x, z_to_y), 1: (x_to_y, x_to_z), 2: (y_to_z, y_to_x) } 
    two_bit_gates = iswap, iiswap, cx, cz, cy, acx, acz, acy
    
    # Nearest-neighbor couplers:
    gateSequence = [ 0, 3, 2, 1, 2, 1, 0, 3 ]
    row_len = math.ceil(math.sqrt(width))
    
    # Cycle through all 3 Pauli axes, every 3 layers
    pauli_axes = [0] * width
    pauli_directions = [0] * width

    for i in range(width):
        # Single bit gates
        for j in range(width):
            # Reset basis, every 3 layers
            pauli_axes[j] = random.randint(0, 2)
            pauli_directions[j] = random.randint(0, 1)
            
            for k in range(3):
                # Sequential basis switch
                gate = single_bit_gates[pauli_axes[j]][pauli_directions[j]]
                if pauli_directions[j]:
                    pauli_axes[j] -= 1
                    if pauli_axes[j] < 0:
                        pauli_axes[j] += 3
                else:
                    pauli_axes[j] += 1
                    if pauli_axes[j] > 2:
                        pauli_axes[j] -= 3

                # Enact random gate choice
                gate_count += gate(circ, j)
                rnd = random.uniform(0, 2 * math.pi)
                circ.mtrx([1, 0, 0, math.cos(rnd) + math.sin(rnd) * 1j], j)
                gate_count = gate_count + 1
            
        # Nearest-neighbor couplers:
        ############################
        gate = gateSequence.pop(0)
        gateSequence.append(gate)
        for row in range(1, row_len, 2):
            for col in range(row_len):
                temp_row = row
                temp_col = col
                temp_row = temp_row + (1 if (gate & 2) else -1);
                temp_col = temp_col + (1 if (gate & 1) else 0)

                if (temp_row < 0) or (temp_col < 0) or (temp_row >= row_len) or (temp_col >= row_len):
                    continue

                b1 = row * row_len + col
                b2 = temp_row * row_len + temp_col

                if (b1 >= width) or (b2 >= width):
                    continue

                g = random.choice(two_bit_gates)
                gate_count += g(circ, b1, b2)

    print("Gate count (before optimization): " + str(gate_count))

    return circ

In [4]:
from pyqrack import QrackSimulator, QrackCircuit

qcircuit = QrackCircuit()
random_circuit(width, qcircuit)
qcircuit.out_to_file('qrack_circuit.qc')

data = ''
with open('qrack_circuit.qc', 'r') as file:
    data = file.read().split()
    
print("Gate count (after optimization): " + str(data[1]))

Gate count (before optimization): 4371
Gate count (after optimization): 514


In [5]:
import time

start = time.perf_counter()
qcircuit = QrackCircuit()
qcircuit.in_from_file('qrack_circuit.qc')
qsim = QrackSimulator()
qcircuit.run(qsim)
print(qsim._qubitCount)
qsim.m_all()
print("Execution seconds: " + str(time.perf_counter() - start))


Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
24
Execution seconds: 0.1139120930001809


If we have Qiskit and numpy installed, we can convert to a Qiskit circuit.

In [6]:
circ = QrackCircuit.file_to_qiskit_circuit('qrack_circuit.qc')
print(circ)

                                                                     »
 q_0: ───────────────────────────────────────────────────────────────»
                                                                     »
 q_1: ───────────────────────────────────────────────────────────────»
                                                     ┌──────────────┐»
 q_2: ───────────────────────────────────────────────┤0             ├»
                                     ┌──────────────┐│              │»
 q_3: ───────────────────────────────┤0             ├┤              ├»
                     ┌──────────────┐│              ││              │»
 q_4: ───────────────┤0             ├┤              ├┤              ├»
      ┌─────────────┐│              ││              ││  Multiplexer │»
 q_5: ┤ Multiplexer ├┤              ├┤              ├┤              ├»
      ├─────────────┤│              ││  Multiplexer ││              │»
 q_6: ┤ Multiplexer ├┤              ├┤              ├┤              ├»
      

In [7]:
from qiskit.compiler.transpiler import transpile

basis_gates = ["u3", "cx"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)
print(circ)

global phase: 2.8776
       ┌────────────────────────────┐      ┌──────────────────────┐»
 q_0: ─┤ U3(2.1439,0.28038,-1.0826) ├───■──┤ U3(π,-2.9717,1.7407) ├»
       └┬─────────────────────────┬─┘   │  └──────────────────────┘»
 q_1: ──┤ U3(2.174,3.1416,1.5708) ├─────┼─────────────■────────────»
      ┌─┴─────────────────────────┴─┐   │             │            »
 q_2: ┤ U3(1.2512,-0.15233,-1.1163) ├───┼─────────────┼────────────»
      ├─────────────────────────────┴┐  │             │            »
 q_3: ┤ U3(3.0634,1.5708,-3.254e-06) ├──┼─────────────┼────────────»
      └─┬──────────────────────────┬─┘  │             │            »
 q_4: ──┤ U3(2.4035,3.1416,1.5708) ├────┼─────────────┼────────────»
      ┌─┴──────────────────────────┴┐ ┌─┴─┐           │            »
 q_5: ┤ U3(2.2035,-0.51516,0.80714) ├─┤ X ├───────────┼────────────»
      └┬────────────────────────────┤ └───┘         ┌─┴─┐          »
 q_6: ─┤ U3(1.1381,2.5989,-0.60759) ├───────────────┤ X ├──────────»
      ┌┴─────

If the basis gates can be converted, a Qiskit `QuantumCircuit` can also be read into a `QrackCircuit`.

In [8]:
qcircuit = QrackCircuit()
qcircuit.in_from_qiskit_circuit(circ)
qcircuit.out_to_file('qiskit.qc')